# MPI: Complexity of collectives and almost collectives

## Recall: Important differences between different point-to-point MPI routines

### Blocking semantics: whether the operands and results are safe to use when the function returns:

- `MPI_Send()` / `MPI_Recv()`: safe to use

- `MPI_Isend()` / `MPI_Irecv()`: wait until `MPI_Request` is complete (`MPI_Wait()` or do other things with `MPI_Test()`)

### Synchronization semantics: has the matching call initiated?

- `MPI_Ssend()` / `MPI_Issend()`: yes!
- `MPI_Send()` / `MPI_Isend()`: not necessarily!
- Clearly when `MPI_Recv()`/`MPI_Irecv()` results are safe to use the matching `MPI_Xsend()`  has initiated :)

### Protocol: how is the message sent?

- **Rendezvous**: only the *envelope* (data type, data size, source, tag, communicator) is send immediately; the destination replies when it is ready
  - Bad for latency (multiple trips in the network), good for memory movement (no additional copies)
- **Eager**: the whole message is sent immediately, stored in an MPI buffer, copied into the destination buffer when the matching `MPI_Recv()` is called
  - Good for latency, but buffer space and memory movement become harder to accomodate with larger message sizes

### One aspect not discussed last time: ordering, progress, and fairness

- **ordering** If `A` sends two messages to `B`, will they be received in that order? Yes!

- **progress** If an `MPI_Send()` on `A` matches an `MPI_Recv()` on `B`, at least one of them will complete:
  - The `MPI_Recv()` could match another message with `MPI_ANY_SOURCE`, so the `MPI_Send()` may not complete always
  - (In multithreading) a different thread at the same process could match the `MPI_Send()`, so the `MPI_Recv()` may not complete always
  
- **fairness** No guarantee! see [this advice](https://www.mpi-forum.org/docs/mpi-3.1/mpi31-report/node58.htm)

## Recall: Collective operations on a communicator

### See the distributed array charts from [Prof. Vuduc's notes](http://vuduc.org/cse6230/slides/cse6230-fa14--06-mpi.pdf)

- `MPI_Barrier()`
- `MPI_Reduce()` $\leftrightarrow$ `MPI_Bcast()`
- `MPI_Gather()` $\leftrightarrow$ `MPI_Scatter()`
  - Varying sizes, `MPI_Gatherv()` $\leftrightarrow$ `MPI_Scatterv()`
- `MPI_Allreduce()`
- `MPI_Allgather()`
  - Varying sizes `MPI_Allgatherv()`
- `MPI_Alltoall()`
  - Varying sizes `MPI_Alltoallv()`
  
### Same questions as with point-to-point: what are the blocking, synchronization, and protocols?

- All of the above are **blocking**.
- Are there non-blocking versions?
  - As of MPI-2, yes! `MPI_Ibarrier()`, etc.
  - Are they useful? Yes, but we have to be careful about order and progress.
  
- `MPI_Barrier()` and the `MPI_All`s are inherently **synchronous**; `MPI_Reduce()`, `MPI_Bcast()`, `MPI_Scatter()`, and `MPI_Gather()` are **not**.

- Like for point-to-point, the **protocols** are system, implementation, and message content dependent!

## Characterizing the real performance of a distributed memory system

Network topology is going to be important for good predictions, but for now we can pretend we are using a totally connected network.

### Last time I mentioned the **LogP** model:

- **L**: the latency between a one byte message being sent and received
- **o**: the process overhead of initiating a `send()`/`recv()`
- **g**: the inverse bandwidth (traditionally, applied to all bytes after the first)
- **P**: the number of nodes in the network

### For an even simple analysis, we can

- combine $L$ and $o$ into the $\lambda$, the latency of a message
- apply **g** to all bytes of a message
- look at the *critical path*: the $R$ dependent messages, communicating total of $W$ words, that maximizes
$\lambda R + gW$

### What are real world estimates of $\lambda$ and $g$?

- [Hager & Wellein, slide 66](https://moodle.rrze.uni-erlangen.de/pluginfile.php/12220/mod_resource/content/10/01_Arch.pdf)

- Using the values from that slide $g$ is four orders of magnitude smaller that $\lambda$
- Recent networks haven't improved that ratio

## Whiteboard exercises: estimating MPI_Collective performance in $\lambda$, $g$, and $P$

- `MPI_Reduce()` an object of size $b$ bytes: $O((\lambda + gb)\log_2 P)$
  - What if we could send and receive $k$ messages simultaneously?
- `MPI_Bcast()`: the same!
- `MPI_Barrier()`: ?
- `MPI_Gather()`: $O(\lambda \log P + gb P)$
- `MPI_Scatter()`: the same!
- `MPI_Allreduce()`: Can we bound in terms of the ones we've already done?
- `MPI_Allgather()`: ?
- `MPI_Alltoall()`: ?

## Example of taking machine parameters into account: CG vs. Pipelined CG

- [Ghysels and Vanroose, 2013](http://dx.doi.org/10.1016/j.parco.2013.06.001)
- Fusing messages together: no advantage in $g$, but less $\lambda$ (which is much larger)

## Example of taking machine parameters and message characteristics into account: Many-to-many

- Process $p$ want to send $k_p$ messages of varying sizes
- No process knows _a priori_ where it will receive messages from
  - Suppose that unknown number for $p$ is $r_p$: what if we don't know it exactly, but we can bound it?
    $$K = \max_p \max (k_p, r_p)$$
    
  - How does $K$ affect our choice of algorithm?
  - If $K \sim P$ (dense) we should build our approach out of collective operations
  - If $K < \log P$ (sparse), we should try to use point-to-point operations where possible
  - Regardless, we can guarantee an $O(\log P)$ lower bound: why?

### Whiteboard: what are some solutions?

- Global exchange
- Global census
  - `MPI_Reduce_scatter` and `MPI_Reduce_scatter_block`!
- Static routing, randomized routing?